In [2093]:
import pandas as pd
import math

ROOT_DATA_DIR = '../in/club_express_exports'
OUT_DATA_DIR = '../out/'

## Read in data
Read in the services data and remove unnecessary columns (Visible? and Notes)

In [2094]:
service_df = pd.read_csv(f"{ROOT_DATA_DIR}/service_data.csv")
visible_services_df = service_df.loc[service_df['Visible?']=='Yes']
trimmed_visible_services_df = visible_services_df.drop(['Visible?', 'Notes'], axis=1)
trimmed_visible_services_df.head()

,Service,Service Category,Transportation?
0,Alarms/Locks/Security,Professional Home/Garden Servi,No
2,Appliance Repair,Professional Home/Garden Servi,No
3,Auto Repair,Professional In-Home Support,No
4,Bill Paying / Paperwork,Volunteer In-Home Support,No
5,Board Meetings,Village Admin,No


Next, we need the request type that each category belongs to. This is relevant because, when creating a type of request (i.e., Transportation Request), the service categories are filtered to only display the categories belonging to the request type.

In [2095]:
completed_and_requested_services_df = pd.read_csv(f"{ROOT_DATA_DIR}/service_provided_service_data.csv")
trimmed_service_info_df = completed_and_requested_services_df[['Service', 'Service Category', 'Request Type']].drop_duplicates()
trimmed_service_info_df

,Service,Service Category,Request Type
0,Gardening/Yard Help - Vol,Volunteer Home/Garden Service,Member's Home
1,PC Help,Technical Support,Member's Home
2,Grocery Shopping,Coronavirus Community Support,Member's Home
3,Office Work - Misc.,Village Admin,Office
4,Handy Person-Vol.,Volunteer Home/Garden Service,Member's Home
...,...,...,...
8085,Answering Phones,Village Admin,Office
8127,FB/SocMedia Admin,Professional In-Home Support,Office
8138,Snow Removal,Professional Home/Garden Servi,Contractor Referral
8154,Dog Trainer/Walker - Professio,Professional In-Home Support,Contractor Referral


## Parse data for H4I Services Dataframe

In [2096]:
# This H4I dataframe will hold all our relevant information.
h4i_service_info_df = pd.DataFrame(columns=['Service', 'Service Category', 'Request Type'])

# Accumulate columns in these lists
services = []
service_categories = []
request_types = []

# Iterate through all current services and insert relevant information into the lists for H4I dataframe columns
for row in trimmed_visible_services_df[['Service', 'Service Category']].iterrows():
    # Get Service, Service Category, and Service Category ID
    service, service_category = row[1].values
    # Get Request Type
    request_type_values = trimmed_service_info_df[trimmed_service_info_df['Service'] == service]['Request Type'].values
    num_request_type_values = len(request_type_values)
    try:
        assert(num_request_type_values <= 1) # Check that there's either 0 or 1 request types associated with this service
    except AssertionError: 
        if "Contractor Referral" in request_type_values:
            list(request_type_values).remove("Contractor Referral")
            num_request_type_values-=1
            if num_request_type_values == 1: # after removing 'Contractor Referral'
                break
        print("ASSERTION ERROR")
        print(request_type_values)
    request_type = None
    if len(request_type_values) == 1:
        request_type = request_type_values[0]
        
    # Append to lists
    services.append(service)
    service_categories.append(service_category)
    request_types.append(request_type)

ASSERTION ERROR
["Member's Home" 'Transportation']


Construct our dataframe with **Service Category** and **Request Type** for every **Service**.

In [2097]:
h4i_service_info_df['Service'] = services
h4i_service_info_df['Service Category'] = service_categories
h4i_service_info_df['Request Type'] = request_types

In [2098]:
h4i_service_info_df.head()

,Service,Service Category,Request Type
0,Alarms/Locks/Security,Professional Home/Garden Servi,None
1,Appliance Repair,Professional Home/Garden Servi,None
2,Auto Repair,Professional In-Home Support,None
3,Bill Paying / Paperwork,Volunteer In-Home Support,Member's Home
4,Board Meetings,Village Admin,Office


## The request type needs to be filled in for some service categories/services.

In [2099]:
h4i_service_info_df.loc[h4i_service_info_df['Request Type'].isnull()]

,Service,Service Category,Request Type
0,Alarms/Locks/Security,Professional Home/Garden Servi,None
1,Appliance Repair,Professional Home/Garden Servi,None
2,Auto Repair,Professional In-Home Support,None
6,Chimneys-Repr.&Clean.,Professional Home/Garden Servi,None
7,Cloud Storage,Technical Support,None
18,Email,Technical Support,None
22,Flooring/Carpets,Professional Home/Garden Servi,None
24,Garden/Landscape-Prof.,Professional Home/Garden Servi,None
27,Grocery Shopping,Coronavirus Community Support,None
28,hack4impact test service,Transportation,None


Fix these rows with "None" as Request Type.

In [2100]:
h4i_service_info_df.loc[[0, 1, 2, 5, 6, 7, 18, 22, 24], 'Request Type']= "Member's Home"

Drop the "hack4impact test service" we created in the system.

In [2101]:
h4i_service_info_df.drop(28, inplace=True); # drop 'hack4impact test service'

Check that there are no more "None" values in Request Type.

In [2102]:
h4i_service_info_df.loc[h4i_service_info_df['Request Type'].isnull()]

,Service,Service Category,Request Type
27,Grocery Shopping,Coronavirus Community Support,None


Remove "Contractor Referral" Requests

In [2103]:
h4i_service_info_df = h4i_service_info_df[h4i_service_info_df['Request Type'] != 'Contractor Referral']
h4i_service_info_df.head()

,Service,Service Category,Request Type
0,Alarms/Locks/Security,Professional Home/Garden Servi,Member's Home
1,Appliance Repair,Professional Home/Garden Servi,Member's Home
2,Auto Repair,Professional In-Home Support,Member's Home
3,Bill Paying / Paperwork,Volunteer In-Home Support,Member's Home
4,Board Meetings,Village Admin,Office


## Create tables of services, service categories, and request types with their IDs

In [2104]:
service_categories_set = list(set(h4i_service_info_df['Service Category']))
services_set = list(set(h4i_service_info_df['Service']))
request_types_set = list(set(h4i_service_info_df['Request Type']))

In [2105]:
h4i_request_types_df = pd.DataFrame()
h4i_request_types_df['Name'] = request_types_set
# h4i_request_types_df.to_csv(f'{OUT_DATA_DIR}/request_types.csv', index_label='ID')
h4i_request_types_df.head()

,Name
0,Transportation
1,None
2,Member's Home
3,Office


In [2106]:
h4i_service_categories_df = pd.DataFrame()
h4i_service_categories_df['Name'] = service_categories_set

# Get Category -> Request Type mappings
category_and_request_type_info = h4i_service_info_df[['Service Category', 'Request Type']].drop_duplicates()
category_and_request_type_info.set_index('Service Category', inplace=True)

# Add request type IDs
request_types = [category_and_request_type_info.loc[category]['Request Type'] for category in h4i_service_categories_df['Name']]

# h4i_service_categories_df.to_csv(f'{OUT_DATA_DIR}/service_categories.csv', index_label='ID')
h4i_service_categories_df.head()

,Name
0,Volunteer Home/Garden Service
1,Technical Support
2,Volunteer In-Home Support
3,Professional Home/Garden Servi
4,Village Admin


In [2107]:
h4i_services_df = pd.DataFrame()
h4i_services_df['Name'] = services_set

# Get Category IDs of services
categories_of_services = [h4i_service_info_df[h4i_service_info_df['Service'] == service]['Service Category'].values[0] for service in h4i_services_df['Name']]                
h4i_services_df['Category ID'] = [h4i_service_categories_df[h4i_service_categories_df['Name'] == category].index[0] for category in categories_of_services]

h4i_services_df.to_csv(f'{OUT_DATA_DIR}/services.csv', index_label='ID')
h4i_services_df.head()

,Name,Category ID
0,Handy Person-Vol.,0
1,Electricians/Electric,3
2,Event Carpool,5
3,Coronavirus Support Phone Call,2
4,Errands w/out Member,2


## Now write the service info df.

In [2108]:
h4i_service_categories_df 

,Name
0,Volunteer Home/Garden Service
1,Technical Support
2,Volunteer In-Home Support
3,Professional Home/Garden Servi
4,Village Admin
5,Transportation
6,Professional In-Home Support
7,Coronavirus Community Support


# People Data

In [2109]:
# Read in member data
members_df = pd.read_csv(f"{ROOT_DATA_DIR}/members_including_metro_area_data.csv")
members_df = members_df.rename(columns={"Address 1" : "Address"})
members_df.head()

,Member Number,First Name,Last Name,Email,Address,Address 2,City,State,Zip,Metro Area,...,Sponsor Name,Chapter,Last Renewal Date,Secondary Type Name,Member Level,Gender,Allow Club Email?,Printed Newsletter?,Mailing Name,Use Alt Address?
0,229,Andrea,Aching,annaching2017@yahoo.com,115 Eddy Street,NaN,Ithaca,NY,14850,NaN,...,NaN,NaN,NaN,Mother,Secondary,Female,Yes,No,Andrea Aching,No
1,228,Gerard,Aching,gerardaching@hotmail.com,115 Eddy Street,NaN,Ithaca,NY,14850,NaN,...,NaN,NaN,NaN,NaN,Primary,Male,Yes,Yes,Gerard L Aching,No
2,230,William,Aching,gerardaching@hotmail.com,115 Eddy Street,NaN,Ithaca,NY,14850,NaN,...,NaN,NaN,NaN,Father,Secondary,Male,Yes,No,NaN,No
3,233,Peggy,Adams,madams@ithaca.edu,417 Hector Street,NaN,Ithaca,NY,14850,Downtown Ithaca,...,NaN,NaN,09/20/2021,NaN,Primary,Female,Yes,No,Margaret Adams,No
4,259,Nancy,Ahlers,baselahlers@gmail.com,2 Horizon Drive,NaN,Ithaca,NY,14850,Lansing,...,NaN,NaN,08/05/2021,NaN,Primary,Female,Yes,No,Nancy Ahlers,No


In [2110]:
# Read in service provider data
service_provider_df = pd.read_csv(f"{ROOT_DATA_DIR}/service_provider_data.csv")
service_provider_df.head()

,Service Provider First Name,Service Provider Laast Name,Type,Company,Address,City,State,Zip,Phone,Cell Phone,Emergency Contact Phone #,Email,Preferred Contact Method,Website,Fully Vetted?,Visible?,Discount Notes
0,NaN,NaN,Non-Member Contractor,AccuFab,232 Cherry St.,Ithaca,NY,14850,(607)273-3706,NaN,NaN,mike@accufabinc.com,Both,http://accufabinc.com/,No,No,NaN
1,NaN,NaN,Non-Member Contractor,Ace Security,720 W. Green St.,Ithaca,NY,14850,6072738840,NaN,NaN,NaN,Phone,NaN,No,No,NaN
2,NaN,NaN,Non-Member Contractor,ACME Pest Control,359 Dryden-Harford Rd.,Dryden,NY,13053,607-844-8689,NaN,NaN,NaN,Phone,NaN,No,No,NaN
3,NaN,NaN,Non-Member Contractor,Austin's Helping Hands LLC,225 S. Fulton St.,Ithaca,NY,14850,6073399321,NaN,NaN,NaN,Phone,NaN,No,No,NaN
4,NaN,NaN,Non-Member Contractor,Bailey Construction,5139 Jacksonville Road,Trumansburg,NY,14886,607-209-4114,NaN,NaN,NaN,Phone,www.baileyconstruction.vpweb.com,No,No,NaN


In [2111]:
# Take out local resource data
local_resource_df = service_provider_df[service_provider_df.Type == 'Non-Member Contractor']
local_resource_df = local_resource_df.drop(['Type','Fully Vetted?', 'Visible?'],axis=1)
local_resource_df = local_resource_df.rename(columns={'Service Provider First Name': 'Contact First Name', 
                                            'Service Provider Laast Name': 'Contact Last Name',
                                           'Emergency Contact Phone #':'Emergency Contact Phone Number',
                                           'Phone': 'Primary Phone Number',
                                           'Cell Phone': 'Secondary Phone Number',
                                            'Discount Notes': 'Notes'})
local_resource_df.head()

,Contact First Name,Contact Last Name,Company,Address,City,State,Zip,Primary Phone Number,Secondary Phone Number,Emergency Contact Phone Number,Email,Preferred Contact Method,Website,Notes
0,NaN,NaN,AccuFab,232 Cherry St.,Ithaca,NY,14850,(607)273-3706,NaN,NaN,mike@accufabinc.com,Both,http://accufabinc.com/,NaN
1,NaN,NaN,Ace Security,720 W. Green St.,Ithaca,NY,14850,6072738840,NaN,NaN,NaN,Phone,NaN,NaN
2,NaN,NaN,ACME Pest Control,359 Dryden-Harford Rd.,Dryden,NY,13053,607-844-8689,NaN,NaN,NaN,Phone,NaN,NaN
3,NaN,NaN,Austin's Helping Hands LLC,225 S. Fulton St.,Ithaca,NY,14850,6073399321,NaN,NaN,NaN,Phone,NaN,NaN
4,NaN,NaN,Bailey Construction,5139 Jacksonville Road,Trumansburg,NY,14886,607-209-4114,NaN,NaN,NaN,Phone,www.baileyconstruction.vpweb.com,NaN


In [2112]:
# Take volunteer data out
volunteer_df = service_provider_df[service_provider_df.Type != 'Non-Member Contractor']
volunteer_df.head()

,Service Provider First Name,Service Provider Laast Name,Type,Company,Address,City,State,Zip,Phone,Cell Phone,Emergency Contact Phone #,Email,Preferred Contact Method,Website,Fully Vetted?,Visible?,Discount Notes
51,Paula,Amols,Member Volunteer,NaN,370 Snyder Hill Rd.,Ithaca,NY,14850,(607) 592-1166,NaN,NaN,pamols54@gmail.com,Email,NaN,Yes,Yes,NaN
52,Wendy,Aquadro,Member Volunteer,NaN,283 ELLIS HOLLOW CREEK RD,ITHACA,NY,14850-9619,6072733049,6073516275,NaN,gsa8@cornell.edu,Phone,NaN,Yes,Yes,NaN
55,Jill,Baldwin,Non-Member Volunteer,Favorite Life Farm,7619 POTTER MORGAN RD,INTERLAKEN,NY,14847-9665,3155214574,3155214574,NaN,jmbaldwin211@gmail.com,Email,NaN,No,No,NaN
57,Eileen,Berlow,Member Volunteer,NaN,2444 Perry City Road,Ithaca,NY,14850,(607) 379-3565,(607) 387-6890,NaN,eileenberlow@yahoo.com,Phone,NaN,No,Yes,NaN
58,Deborah,Berman,Non-Member Volunteer,NaN,110 Hancock St,Ithaca,NY,14850,(607) 273-2901,NaN,(607) 342-7841,skaboooch@gmail.com,Email,NaN,Yes,Yes,NaN


## Address ID Replacement

In [2113]:
# Add volunteer and local resource addresses to address book
column_names = ['Address','City','State','Zip']
address_data = []
for idx, row in members_df.iterrows():
    d = {}
    for c in column_names:
        d[c] = row[c]
    d['Name'] = row['First Name'] + " " + row['Last Name']
    address_data.append(d)
for idx, row in local_resource_df.iterrows():
    d = {}
    for c in column_names:
        d[c] = row[c]
    d['Name'] = row['Company']
    address_data.append(d)
for idx, row in volunteer_df.iterrows():
    d = {}
    for c in column_names:
        d[c] = row[c]
    d['Name'] = row['Service Provider First Name'] + " " + row['Service Provider Laast Name']
    address_data.append(d)
address_df = pd.DataFrame(address_data)
address_df = address_df.drop_duplicates()
address_df = address_df.reset_index(drop=True)
address_df = address_df.reset_index()
address_df = address_df.rename(columns={'index': 'Address ID'})
address_df.head()

,Address ID,Address,City,State,Zip,Name
0,0,115 Eddy Street,Ithaca,NY,14850,Andrea Aching
1,1,115 Eddy Street,Ithaca,NY,14850,Gerard Aching
2,2,115 Eddy Street,Ithaca,NY,14850,William Aching
3,3,417 Hector Street,Ithaca,NY,14850,Peggy Adams
4,4,2 Horizon Drive,Ithaca,NY,14850,Nancy Ahlers


In [2114]:
address_df = address_df[["Address ID", "Name", "Address", "City", "Zip"]]
address_df.rename(columns={"Address ID": "id", "Name": "name", "Address": "address1", "City" : "city", "Zip" : "zipcode"}, inplace=True)
address_df = address_df[address_df['name'].notna()]
address_df = address_df[address_df['address1'].notna()]
address_df.head()

,id,name,address1,city,zipcode
0,0,Andrea Aching,115 Eddy Street,Ithaca,14850
1,1,Gerard Aching,115 Eddy Street,Ithaca,14850
2,2,William Aching,115 Eddy Street,Ithaca,14850
3,3,Peggy Adams,417 Hector Street,Ithaca,14850
4,4,Nancy Ahlers,2 Horizon Drive,Ithaca,14850


In [2115]:
# Export addresses
address_df.to_csv(OUT_DATA_DIR+'addresses.csv', index=False)

In [2116]:
# Replace addresses in members with address ID
volunteer_df = volunteer_df.drop(['Address','City','State','Zip'], axis=1)
local_resource_df = local_resource_df.drop(['Address','City','State','Zip'], axis=1)
members_df = members_df.drop(['Address','City','State','Zip'], axis=1)
volunteer_df.head()

,Service Provider First Name,Service Provider Laast Name,Type,Company,Phone,Cell Phone,Emergency Contact Phone #,Email,Preferred Contact Method,Website,Fully Vetted?,Visible?,Discount Notes
51,Paula,Amols,Member Volunteer,NaN,(607) 592-1166,NaN,NaN,pamols54@gmail.com,Email,NaN,Yes,Yes,NaN
52,Wendy,Aquadro,Member Volunteer,NaN,6072733049,6073516275,NaN,gsa8@cornell.edu,Phone,NaN,Yes,Yes,NaN
55,Jill,Baldwin,Non-Member Volunteer,Favorite Life Farm,3155214574,3155214574,NaN,jmbaldwin211@gmail.com,Email,NaN,No,No,NaN
57,Eileen,Berlow,Member Volunteer,NaN,(607) 379-3565,(607) 387-6890,NaN,eileenberlow@yahoo.com,Phone,NaN,No,Yes,NaN
58,Deborah,Berman,Non-Member Volunteer,NaN,(607) 273-2901,NaN,(607) 342-7841,skaboooch@gmail.com,Email,NaN,Yes,Yes,NaN


In [2117]:
members_df.head()

,Member Number,First Name,Last Name,Email,Address 2,Metro Area,Country,Phone,Cell Phone,Fax,...,Sponsor Name,Chapter,Last Renewal Date,Secondary Type Name,Member Level,Gender,Allow Club Email?,Printed Newsletter?,Mailing Name,Use Alt Address?
0,229,Andrea,Aching,annaching2017@yahoo.com,NaN,NaN,United States of America,6073194170,7073383665,NaN,...,NaN,NaN,NaN,Mother,Secondary,Female,Yes,No,Andrea Aching,No
1,228,Gerard,Aching,gerardaching@hotmail.com,NaN,NaN,United States of America,6073194170,9175455737,NaN,...,NaN,NaN,NaN,NaN,Primary,Male,Yes,Yes,Gerard L Aching,No
2,230,William,Aching,gerardaching@hotmail.com,NaN,NaN,United States of America,6073194170,NaN,NaN,...,NaN,NaN,NaN,Father,Secondary,Male,Yes,No,NaN,No
3,233,Peggy,Adams,madams@ithaca.edu,NaN,Downtown Ithaca,United States of America,6078821659,6078821659,NaN,...,NaN,NaN,09/20/2021,NaN,Primary,Female,Yes,No,Margaret Adams,No
4,259,Nancy,Ahlers,baselahlers@gmail.com,NaN,Lansing,United States of America,NaN,NaN,NaN,...,NaN,NaN,08/05/2021,NaN,Primary,Female,Yes,No,Nancy Ahlers,No


In [2118]:
# Drop irrelevant columns from members_df and rename as necessary
members_df = members_df[['Member Number','First Name', 'Middle Initial','Last Name',
                        'Gender','Nickname', 'Phone', 'Cell Phone',
                        'Email']]
members_df = members_df.rename(columns={"Nickname": "Preferred Name", "Phone": "Primary Phone Number", 
                           "Cell Phone": "Secondary Phone Number", "Email": "Email Address"})
members_df['Salutation'] = None
members_df['Volunteer Id'] = None
members_df['Primary Address Id'] = None
members_df['Secondary Address Id'] = None
members_df['Preferred Contact Method']  = None
members_df.head()

,Member Number,First Name,Middle Initial,Last Name,Gender,Preferred Name,Primary Phone Number,Secondary Phone Number,Email Address,Salutation,Volunteer Id,Primary Address Id,Secondary Address Id,Preferred Contact Method
0,229,Andrea,NaN,Aching,Female,NaN,6073194170,7073383665,annaching2017@yahoo.com,None,None,None,None,None
1,228,Gerard,L,Aching,Male,NaN,6073194170,9175455737,gerardaching@hotmail.com,None,None,None,None,None
2,230,William,NaN,Aching,Male,NaN,6073194170,NaN,gerardaching@hotmail.com,None,None,None,None,None
3,233,Peggy,NaN,Adams,Female,NaN,6078821659,6078821659,madams@ithaca.edu,None,None,None,None,None
4,259,Nancy,NaN,Ahlers,Female,NaN,NaN,NaN,baselahlers@gmail.com,None,None,None,None,None


In [2119]:
# Read in emergency contact information and rename columns
members_emergency_contact = pd.read_csv(f"{ROOT_DATA_DIR}/member_emergency_contact.csv")
members_emergency_contact = members_emergency_contact.rename(columns={"Contact Name": "Emergency Contact Name", 
                                                                      "Relationship": "Emergency Contact Relationship", 
                                                                      "Contact Email": "Emergency Contact Email Address",
                                                                     "Contact Phone": "Emergency Contact Phone Number"})
members_emergency_contact.head()

,Member Name,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Email Address,Emergency Contact Phone Number
0,Peggy Adams,NaN,NaN,NaN,NaN
1,Nancy Ahlers,NaN,NaN,NaN,NaN
2,Dilmeran Akgoze,Anita Racine,close friend and retired from Cornell,NaN,607-273-1235
3,Paula Amols,NaN,NaN,NaN,NaN
4,Wendy Aquadro,NaN,NaN,NaN,NaN


In [2120]:
# Add member emergency contact information
members_df['Member Name'] = members_df['First Name'] + ' ' + members_df['Last Name']
members_df = members_df.merge(members_emergency_contact, how='left',
                 left_on='Member Name', right_on='Member Name')
members_df.drop(['Member Name'],axis=1)
members_df.head()

,Member Number,First Name,Middle Initial,Last Name,Gender,Preferred Name,Primary Phone Number,Secondary Phone Number,Email Address,Salutation,Volunteer Id,Primary Address Id,Secondary Address Id,Preferred Contact Method,Member Name,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Email Address,Emergency Contact Phone Number
0,229,Andrea,NaN,Aching,Female,NaN,6073194170,7073383665,annaching2017@yahoo.com,None,None,None,None,None,Andrea Aching,NaN,NaN,NaN,NaN
1,228,Gerard,L,Aching,Male,NaN,6073194170,9175455737,gerardaching@hotmail.com,None,None,None,None,None,Gerard Aching,NaN,NaN,NaN,NaN
2,230,William,NaN,Aching,Male,NaN,6073194170,NaN,gerardaching@hotmail.com,None,None,None,None,None,William Aching,NaN,NaN,NaN,NaN
3,233,Peggy,NaN,Adams,Female,NaN,6078821659,6078821659,madams@ithaca.edu,None,None,None,None,None,Peggy Adams,NaN,NaN,NaN,NaN
4,259,Nancy,NaN,Ahlers,Female,NaN,NaN,NaN,baselahlers@gmail.com,None,None,None,None,None,Nancy Ahlers,NaN,NaN,NaN,NaN


In [2121]:
# Read in member notes information and preprocess out irrelevant columns, rename relevant columns
members_notes = pd.read_csv(f"{ROOT_DATA_DIR}/member_notes.csv")
members_notes.reset_index(inplace=True)
members_notes = members_notes.drop(['level_1', 'Name', 'Phone'], axis=1)
members_notes = members_notes.rename(columns={'level_0': 'Member Name', 'Email':'Staffer Notes'})
members_notes["Volunteer Notes"] = ""
members_notes.head()

,Member Name,Staffer Notes,Volunteer Notes
0,"Dubovi, Robin","Prefers not to drive in snowy, bad-weather con...",
1,"Lemley, Ann",Committee Member Only,
2,"Cathles, Mary Helen",Larry Cathles spouse\r\n607-533-7589 (h)\r\n60...,
3,"Regenstein, Carrie",Emergency Contact: \r\nJoe Regenstein husband\...,
4,"Jaquette, John","Emergency Contact is Susan Jaquette, wife\r\ne...",


In [2122]:
# Add member notes to members df
members_df = members_df.merge(members_notes, how='left',
                 left_on='Member Name', right_on='Member Name',)
members_df.head()

,Member Number,First Name,Middle Initial,Last Name,Gender,Preferred Name,Primary Phone Number,Secondary Phone Number,Email Address,Salutation,...,Primary Address Id,Secondary Address Id,Preferred Contact Method,Member Name,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Email Address,Emergency Contact Phone Number,Staffer Notes,Volunteer Notes
0,229,Andrea,NaN,Aching,Female,NaN,6073194170,7073383665,annaching2017@yahoo.com,None,...,None,None,None,Andrea Aching,NaN,NaN,NaN,NaN,NaN,NaN
1,228,Gerard,L,Aching,Male,NaN,6073194170,9175455737,gerardaching@hotmail.com,None,...,None,None,None,Gerard Aching,NaN,NaN,NaN,NaN,NaN,NaN
2,230,William,NaN,Aching,Male,NaN,6073194170,NaN,gerardaching@hotmail.com,None,...,None,None,None,William Aching,NaN,NaN,NaN,NaN,NaN,NaN
3,233,Peggy,NaN,Adams,Female,NaN,6078821659,6078821659,madams@ithaca.edu,None,...,None,None,None,Peggy Adams,NaN,NaN,NaN,NaN,NaN,NaN
4,259,Nancy,NaN,Ahlers,Female,NaN,NaN,NaN,baselahlers@gmail.com,None,...,None,None,None,Nancy Ahlers,NaN,NaN,NaN,NaN,NaN,NaN


In [2123]:
# Now drop the extra column we created
members_df.drop(['Member Name'],axis=1)

,Member Number,First Name,Middle Initial,Last Name,Gender,Preferred Name,Primary Phone Number,Secondary Phone Number,Email Address,Salutation,Volunteer Id,Primary Address Id,Secondary Address Id,Preferred Contact Method,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Email Address,Emergency Contact Phone Number,Staffer Notes,Volunteer Notes
0,229,Andrea,NaN,Aching,Female,NaN,6073194170,7073383665,annaching2017@yahoo.com,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,228,Gerard,L,Aching,Male,NaN,6073194170,9175455737,gerardaching@hotmail.com,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,230,William,NaN,Aching,Male,NaN,6073194170,NaN,gerardaching@hotmail.com,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,233,Peggy,NaN,Adams,Female,NaN,6078821659,6078821659,madams@ithaca.edu,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,259,Nancy,NaN,Ahlers,Female,NaN,NaN,NaN,baselahlers@gmail.com,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,420,Linda,NaN,Woodard,Female,NaN,6072279798,NaN,lab6@cornell.edu,None,None,None,None,None,Allison Pfuntner,Daughter,allisonpeters3@yahoo.com,301-503-1123,NaN,NaN
326,234,Rochelle,NaN,Woods,Female,NaN,607-277-3497,607-339-1956,rewoods112@gmail.com,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
327,43,Eugene,NaN,Yarussi,Male,Gene,(607) 277-6520,(607) 279-6613,eyarussi@gmail.com,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
328,44,Jeanne,NaN,Yarussi,Female,NaN,(607) 277-6520,(607) 279-6613,eyarussi@gmail.com,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [2124]:
# Create an ID column
members_df['ID'] = range(0, len(members_df))
members_df.head()

,Member Number,First Name,Middle Initial,Last Name,Gender,Preferred Name,Primary Phone Number,Secondary Phone Number,Email Address,Salutation,...,Secondary Address Id,Preferred Contact Method,Member Name,Emergency Contact Name,Emergency Contact Relationship,Emergency Contact Email Address,Emergency Contact Phone Number,Staffer Notes,Volunteer Notes,ID
0,229,Andrea,NaN,Aching,Female,NaN,6073194170,7073383665,annaching2017@yahoo.com,None,...,None,None,Andrea Aching,NaN,NaN,NaN,NaN,NaN,NaN,0
1,228,Gerard,L,Aching,Male,NaN,6073194170,9175455737,gerardaching@hotmail.com,None,...,None,None,Gerard Aching,NaN,NaN,NaN,NaN,NaN,NaN,1
2,230,William,NaN,Aching,Male,NaN,6073194170,NaN,gerardaching@hotmail.com,None,...,None,None,William Aching,NaN,NaN,NaN,NaN,NaN,NaN,2
3,233,Peggy,NaN,Adams,Female,NaN,6078821659,6078821659,madams@ithaca.edu,None,...,None,None,Peggy Adams,NaN,NaN,NaN,NaN,NaN,NaN,3
4,259,Nancy,NaN,Ahlers,Female,NaN,NaN,NaN,baselahlers@gmail.com,None,...,None,None,Nancy Ahlers,NaN,NaN,NaN,NaN,NaN,NaN,4


In [2125]:
members_df = members_df[["ID", "Salutation", "First Name", "Middle Initial", "Last Name", "Preferred Name", "Gender", "Volunteer Id", "Member Number", "Primary Address Id", "Secondary Address Id", "Primary Phone Number", "Secondary Phone Number", "Email Address", "Preferred Contact Method", "Emergency Contact Name", "Emergency Contact Phone Number", "Emergency Contact Email Address", "Emergency Contact Relationship", "Volunteer Notes", "Staffer Notes"]]
members_df.rename(columns={"ID": "id", "Salutation": "salutation", "First Name": "first_name", "Middle Initial" : "middle_initial", "Last Name" : "last_name", "Preferred Name" : "preferred_name", "Gender" : "gender", "Volunteer Id" : "volunteer_id", "Member Number" : "member_number", "Primary Address Id" : "primary_address_id", "Secondary Address Id" : "secondary_address_id", "Primary Phone Number": "primary_phone_number", "Secondary Phone Number" : "secondary_phone_number", "Email Address" : "email_address", "Preferred Contact Method" : "preferred_contact_method", "Emergency Contact Name" : "emergency_contact_name", "Emergency Contact Phone Number" : "emergency_contact_phone_number", "Emergency Contact Email Address" : "emergency_contact_email_address", "Emergency Contact Relationship" : "emergency_contact_relationship", "Volunteer Notes" : "volunteer_notes", "Staffer Notes" : "staffer_notes"}, inplace=True)
members_df.head()

,id,salutation,first_name,middle_initial,last_name,preferred_name,gender,volunteer_id,member_number,primary_address_id,...,primary_phone_number,secondary_phone_number,email_address,preferred_contact_method,emergency_contact_name,emergency_contact_phone_number,emergency_contact_email_address,emergency_contact_relationship,volunteer_notes,staffer_notes
0,0,None,Andrea,NaN,Aching,NaN,Female,None,229,None,...,6073194170,7073383665,annaching2017@yahoo.com,None,NaN,NaN,NaN,NaN,NaN,NaN
1,1,None,Gerard,L,Aching,NaN,Male,None,228,None,...,6073194170,9175455737,gerardaching@hotmail.com,None,NaN,NaN,NaN,NaN,NaN,NaN
2,2,None,William,NaN,Aching,NaN,Male,None,230,None,...,6073194170,NaN,gerardaching@hotmail.com,None,NaN,NaN,NaN,NaN,NaN,NaN
3,3,None,Peggy,NaN,Adams,NaN,Female,None,233,None,...,6078821659,6078821659,madams@ithaca.edu,None,NaN,NaN,NaN,NaN,NaN,NaN
4,4,None,Nancy,NaN,Ahlers,NaN,Female,None,259,None,...,NaN,NaN,baselahlers@gmail.com,None,NaN,NaN,NaN,NaN,NaN,NaN


In [2126]:
list(members_df.columns)

['id',
 'salutation',
 'first_name',
 'middle_initial',
 'last_name',
 'preferred_name',
 'gender',
 'volunteer_id',
 'member_number',
 'primary_address_id',
 'secondary_address_id',
 'primary_phone_number',
 'secondary_phone_number',
 'email_address',
 'preferred_contact_method',
 'emergency_contact_name',
 'emergency_contact_phone_number',
 'emergency_contact_email_address',
 'emergency_contact_relationship',
 'volunteer_notes',
 'staffer_notes']

In [2127]:
list(volunteer_df.columns)

['Service Provider First Name',
 'Service Provider Laast Name',
 'Type',
 'Company',
 'Phone',
 'Cell Phone',
 'Emergency Contact Phone #',
 'Email',
 'Preferred Contact Method',
 'Website',
 'Fully Vetted?',
 'Visible?',
 'Discount Notes']

In [2128]:
# Preprocess volunteer data by dropping irrelevant data, renaming columns.
volunteer_df = volunteer_df.drop(['Type', 'Company', 'Website','Visible?','Discount Notes'],axis=1)
volunteer_df = volunteer_df.rename(columns={'Service Provider First Name': 'First Name', 
                                            'Service Provider Laast Name': 'Last Name',
                                           'Emergency Contact Phone #':'Emergency Contact Phone Number',
                                           'Fully Vetted?': 'Vetting',
                                           'Phone': 'Primary Phone Number',
                                           'Cell Phone': 'Secondary Phone Number',
                                           "Email" : "Email Address"})
volunteer_df['General Notes'] = None
volunteer_df.head()

,First Name,Last Name,Primary Phone Number,Secondary Phone Number,Emergency Contact Phone Number,Email Address,Preferred Contact Method,Vetting,General Notes
51,Paula,Amols,(607) 592-1166,NaN,NaN,pamols54@gmail.com,Email,Yes,None
52,Wendy,Aquadro,6072733049,6073516275,NaN,gsa8@cornell.edu,Phone,Yes,None
55,Jill,Baldwin,3155214574,3155214574,NaN,jmbaldwin211@gmail.com,Email,No,None
57,Eileen,Berlow,(607) 379-3565,(607) 387-6890,NaN,eileenberlow@yahoo.com,Phone,No,None
58,Deborah,Berman,(607) 273-2901,NaN,(607) 342-7841,skaboooch@gmail.com,Email,Yes,None


In [2129]:
volunteer_notes = pd.read_csv(f"{ROOT_DATA_DIR}/service_provider_vettings.csv")
volunteer_notes.reset_index(inplace=True)
volunteer_notes = volunteer_notes.rename(columns={'Additional Data':'Vetting Notes', "index":"Member Name"})
volunteer_notes = volunteer_notes.drop(['Fully Vetted?', 'Type', 'Expiration', "Name"], axis=1)
names = volunteer_notes["Member Name"].unique()
volunteer_notes = volunteer_notes.loc[volunteer_notes["Member Name"].notnull() ]

for name in names:
  notes = volunteer_notes[volunteer_notes["Member Name"] == name]['Vetting Notes'].unique()
  volunteer_notes = volunteer_notes.loc[volunteer_notes["Member Name"] != name ]
  volunteer_notes.loc[len(volunteer_notes.index)] = [name,(notes)] 


volunteer_df['Member Name'] = volunteer_df['First Name'] + ' '+ volunteer_df['Last Name']
volunteer_df = volunteer_df.merge(volunteer_notes, how='left',
                 left_on='Member Name', right_on='Member Name')
volunteer_df.drop(['Member Name'],axis=1)
volunteer_df.head()

/Users/ericfreitag/Documents/Hack4Impact/LLH/love-living-at-home/venv/lib/python3.9/site-packages/pandas/core/internals/blocks.py:983: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.array(value)


,First Name,Last Name,Primary Phone Number,Secondary Phone Number,Emergency Contact Phone Number,Email Address,Preferred Contact Method,Vetting,General Notes,Member Name,Vetting Notes
0,Paula,Amols,(607) 592-1166,NaN,NaN,pamols54@gmail.com,Email,Yes,None,Paula Amols,"[nan, She is a very capable person who is hone..."
1,Wendy,Aquadro,6072733049,6073516275,NaN,gsa8@cornell.edu,Phone,Yes,None,Wendy Aquadro,NaN
2,Jill,Baldwin,3155214574,3155214574,NaN,jmbaldwin211@gmail.com,Email,No,None,Jill Baldwin,NaN
3,Eileen,Berlow,(607) 379-3565,(607) 387-6890,NaN,eileenberlow@yahoo.com,Phone,No,None,Eileen Berlow,NaN
4,Deborah,Berman,(607) 273-2901,NaN,(607) 342-7841,skaboooch@gmail.com,Email,Yes,None,Deborah Berman,"[nan, thinks highly of Deborah., Is good with ..."


In [2130]:
volunteer_df.insert(1,"Middle Initial", None, True)
volunteer_df.insert(0,"Salutation", None, True)
volunteer_df.insert(4, "Preferred Name", None, True)
volunteer_df.insert(5,"Gender", None, True)
volunteer_df.insert(7,"Member Id", None, True)
volunteer_df.head()

,Salutation,First Name,Middle Initial,Last Name,Preferred Name,Gender,Primary Phone Number,Member Id,Secondary Phone Number,Emergency Contact Phone Number,Email Address,Preferred Contact Method,Vetting,General Notes,Member Name,Vetting Notes
0,None,Paula,None,Amols,None,None,(607) 592-1166,None,NaN,NaN,pamols54@gmail.com,Email,Yes,None,Paula Amols,"[nan, She is a very capable person who is hone..."
1,None,Wendy,None,Aquadro,None,None,6072733049,None,6073516275,NaN,gsa8@cornell.edu,Phone,Yes,None,Wendy Aquadro,NaN
2,None,Jill,None,Baldwin,None,None,3155214574,None,3155214574,NaN,jmbaldwin211@gmail.com,Email,No,None,Jill Baldwin,NaN
3,None,Eileen,None,Berlow,None,None,(607) 379-3565,None,(607) 387-6890,NaN,eileenberlow@yahoo.com,Phone,No,None,Eileen Berlow,NaN
4,None,Deborah,None,Berman,None,None,(607) 273-2901,None,NaN,(607) 342-7841,skaboooch@gmail.com,Email,Yes,None,Deborah Berman,"[nan, thinks highly of Deborah., Is good with ..."


In [2131]:
# Create an ID column
volunteer_df.insert(0,"ID", range(0, len(volunteer_df)), True)
volunteer_df.head()

,ID,Salutation,First Name,Middle Initial,Last Name,Preferred Name,Gender,Primary Phone Number,Member Id,Secondary Phone Number,Emergency Contact Phone Number,Email Address,Preferred Contact Method,Vetting,General Notes,Member Name,Vetting Notes
0,0,None,Paula,None,Amols,None,None,(607) 592-1166,None,NaN,NaN,pamols54@gmail.com,Email,Yes,None,Paula Amols,"[nan, She is a very capable person who is hone..."
1,1,None,Wendy,None,Aquadro,None,None,6072733049,None,6073516275,NaN,gsa8@cornell.edu,Phone,Yes,None,Wendy Aquadro,NaN
2,2,None,Jill,None,Baldwin,None,None,3155214574,None,3155214574,NaN,jmbaldwin211@gmail.com,Email,No,None,Jill Baldwin,NaN
3,3,None,Eileen,None,Berlow,None,None,(607) 379-3565,None,(607) 387-6890,NaN,eileenberlow@yahoo.com,Phone,No,None,Eileen Berlow,NaN
4,4,None,Deborah,None,Berman,None,None,(607) 273-2901,None,NaN,(607) 342-7841,skaboooch@gmail.com,Email,Yes,None,Deborah Berman,"[nan, thinks highly of Deborah., Is good with ..."


In [2132]:
volunteer_df['Emergency Contact Name'] = None
volunteer_df['Emergency Contact Email Address'] = None
volunteer_df['Emergency Contact Relationship'] = None
volunteer_df['Primary Address Id'] = None
volunteer_df['Secondary Address Id'] = None
volunteer_df.insert(0,"Availability Id", range(1, len(volunteer_df) + 1), True)
list(volunteer_df.columns)

/Users/ericfreitag/Documents/Hack4Impact/LLH/love-living-at-home/venv/lib/python3.9/site-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


['Availability Id',
 'ID',
 'Salutation',
 'First Name',
 'Middle Initial',
 'Last Name',
 'Preferred Name',
 'Gender',
 'Primary Phone Number',
 'Member Id',
 'Secondary Phone Number',
 'Emergency Contact Phone Number',
 'Email Address',
 'Preferred Contact Method',
 'Vetting',
 'General Notes',
 'Member Name',
 'Vetting Notes',
 'Emergency Contact Name',
 'Emergency Contact Email Address',
 'Emergency Contact Relationship',
 'Primary Address Id',
 'Secondary Address Id']

In [2133]:
volunteer_df = volunteer_df[["ID", "Salutation", "First Name", "Middle Initial", "Last Name", "Preferred Name", "Gender", "Member Id", "Primary Address Id", "Secondary Address Id", "Primary Phone Number", "Secondary Phone Number", "Email Address", "Preferred Contact Method", "Emergency Contact Name", "Emergency Contact Phone Number", "Emergency Contact Email Address", "Emergency Contact Relationship", "Vetting Notes", "Availability Id", "General Notes"]]
volunteer_df.rename(columns={"ID": "id", "Salutation": "salutation", "First Name": "first_name", "Middle Initial" : "middle_initial", "Last Name" : "last_name", "Preferred Name" : "preferred_name", "Gender" : "gender", "Member Id" : "member_id", "Primary Address Id" : "primary_address_id", "Secondary Address Id" : "secondary_address_id", "Primary Phone Number": "primary_phone_number", "Secondary Phone Number" : "secondary_phone_number", "Email Address" : "email_address", "Preferred Contact Method" : "preferred_contact_method", "Emergency Contact Name" : "emergency_contact_name", "Emergency Contact Phone Number" : "emergency_contact_phone_number", "Emergency Contact Email Address" : "emergency_contact_email_address", "Emergency Contact Relationship" : "emergency_contact_relationship", "Vetting Notes" : "vetting_notes", "Availability Id" : "availability_id", "General Notes" : "general_notes"}, inplace=True)
volunteer_df.head()

,id,salutation,first_name,middle_initial,last_name,preferred_name,gender,member_id,primary_address_id,secondary_address_id,...,secondary_phone_number,email_address,preferred_contact_method,emergency_contact_name,emergency_contact_phone_number,emergency_contact_email_address,emergency_contact_relationship,vetting_notes,availability_id,general_notes
0,0,None,Paula,None,Amols,None,None,None,None,None,...,NaN,pamols54@gmail.com,Email,None,NaN,None,None,"[nan, She is a very capable person who is hone...",1,None
1,1,None,Wendy,None,Aquadro,None,None,None,None,None,...,6073516275,gsa8@cornell.edu,Phone,None,NaN,None,None,NaN,2,None
2,2,None,Jill,None,Baldwin,None,None,None,None,None,...,3155214574,jmbaldwin211@gmail.com,Email,None,NaN,None,None,NaN,3,None
3,3,None,Eileen,None,Berlow,None,None,None,None,None,...,(607) 387-6890,eileenberlow@yahoo.com,Phone,None,NaN,None,None,NaN,4,None
4,4,None,Deborah,None,Berman,None,None,None,None,None,...,NaN,skaboooch@gmail.com,Email,None,(607) 342-7841,None,None,"[nan, thinks highly of Deborah., Is good with ...",5,None


In [2134]:
list(volunteer_df.columns)

['id',
 'salutation',
 'first_name',
 'middle_initial',
 'last_name',
 'preferred_name',
 'gender',
 'member_id',
 'primary_address_id',
 'secondary_address_id',
 'primary_phone_number',
 'secondary_phone_number',
 'email_address',
 'preferred_contact_method',
 'emergency_contact_name',
 'emergency_contact_phone_number',
 'emergency_contact_email_address',
 'emergency_contact_relationship',
 'vetting_notes',
 'availability_id',
 'general_notes']

## Metro Areas

In [2135]:
# Create and save metro area df
metro_areas = pd.DataFrame(['Brooktondale', 'Caroline', 'Danby', 'Downtown Ithaca', 'Dryden', 'Enfield', 'Freeville', 'Groton', 'Lansing', 'Newfield', 'Outside Tompkins County', 'Slaterville Springs', 'South Hill', 'Tompkins County', 'Trumansburg', 'Jacksonville', 'Ulysses', 'Vana', 'West Hill'])
metro_areas.to_csv(OUT_DATA_DIR+'metro_areas.csv')

In [2136]:
# Export people data
members_df.to_csv(OUT_DATA_DIR+'members.csv', index=False)
volunteer_df.to_csv(OUT_DATA_DIR+'volunteers.csv', index=False)
local_resource_df.to_csv(OUT_DATA_DIR+'local_resources.csv')

## Clean Local Resource Data

In [2137]:
local_resources_df = pd.read_csv('../out/archive/local_resources.csv')

In [2138]:
local_resources_df.head()

,Unnamed: 0,Contact First Name,Contact Last Name,Company,Primary Phone Number,Secondary Phone Number,Emergency Contact Phone Number,Email,Preferred Contact Method,Website,Notes,Address ID
0,0,NaN,NaN,Ace Security,401.0,NaN,NaN,NaN,Phone,NaN,NaN,197
1,1,NaN,NaN,ACME Pest Control,402.0,NaN,NaN,NaN,Phone,NaN,NaN,198
2,2,NaN,NaN,Bailey Construction,403.0,NaN,NaN,NaN,Phone,www.baileyconstruction.vpweb.com,NaN,199
3,3,NaN,NaN,Bell's Auto Service,404.0,NaN,NaN,NaN,Phone,www.bellsauto.com,NaN,200
4,4,NaN,NaN,Bumblebee Painters,405.0,NaN,NaN,info@bumblebeepainters.com,Email,www.bumblebeepainters.com,NaN,201


Create an 'ID' column.

In [2139]:
local_resources_df.rename(columns={local_resources_df.columns[0]: "ID"}, inplace=True)
local_resources_df.head()

,ID,Contact First Name,Contact Last Name,Company,Primary Phone Number,Secondary Phone Number,Emergency Contact Phone Number,Email,Preferred Contact Method,Website,Notes,Address ID
0,0,NaN,NaN,Ace Security,401.0,NaN,NaN,NaN,Phone,NaN,NaN,197
1,1,NaN,NaN,ACME Pest Control,402.0,NaN,NaN,NaN,Phone,NaN,NaN,198
2,2,NaN,NaN,Bailey Construction,403.0,NaN,NaN,NaN,Phone,www.baileyconstruction.vpweb.com,NaN,199
3,3,NaN,NaN,Bell's Auto Service,404.0,NaN,NaN,NaN,Phone,www.bellsauto.com,NaN,200
4,4,NaN,NaN,Bumblebee Painters,405.0,NaN,NaN,info@bumblebeepainters.com,Email,www.bumblebeepainters.com,NaN,201


Read in all the phone numbers

In [2140]:
phone_df = pd.read_csv('../out/phone_numbers.csv')
phone_df.columns

Index(['Unnamed: 0', 'Phone ID', 'Phone Number'], dtype='object')

In [2141]:
phone_df.drop(columns=['Unnamed: 0', 'Phone ID'], inplace=True)

In [2142]:
local_resources_df['Primary Phone Number Text'] = [phone_df.iloc[int(phone_id)]['Phone Number'] if not math.isnan(phone_id) else None for phone_id in local_resources_df['Primary Phone Number']]
local_resources_df['Secondary Phone Number Text'] = [phone_df.iloc[int(phone_id)]['Phone Number'] if not math.isnan(phone_id) else None for phone_id in local_resources_df['Secondary Phone Number']]

In [2143]:
local_resources_df.drop(columns=['Primary Phone Number', 'Secondary Phone Number'], inplace=True)
local_resources_df.rename(columns={'Primary Phone Number Text': 'Primary Phone Number', 'Secondary Phone Number Text': 'Secondary Phone Number'}, inplace=True)

In [2144]:
local_resources_df.head()

,ID,Contact First Name,Contact Last Name,Company,Emergency Contact Phone Number,Email,Preferred Contact Method,Website,Notes,Address ID,Primary Phone Number,Secondary Phone Number
0,0,NaN,NaN,Ace Security,NaN,NaN,Phone,NaN,NaN,197,6072738840,None
1,1,NaN,NaN,ACME Pest Control,NaN,NaN,Phone,NaN,NaN,198,607-844-8689,None
2,2,NaN,NaN,Bailey Construction,NaN,NaN,Phone,www.baileyconstruction.vpweb.com,NaN,199,607-209-4114,None
3,3,NaN,NaN,Bell's Auto Service,NaN,NaN,Phone,www.bellsauto.com,NaN,200,607-273-9325,None
4,4,NaN,NaN,Bumblebee Painters,NaN,info@bumblebeepainters.com,Email,www.bumblebeepainters.com,NaN,201,607-273-6521,None


## More reorganizing

All the emergency contact phone numbers are `NaN` in this table, so we'll remove the column.

In [2145]:
set([math.isnan(value) for value in local_resources_df['Emergency Contact Phone Number'].values])

{True}

In [2146]:
local_resources_df.drop(columns=['Emergency Contact Phone Number'], inplace=True)

In [2147]:
local_resources_df.columns

Index(['ID', 'Contact First Name', 'Contact Last Name', 'Company', 'Email',
       'Preferred Contact Method', 'Website', 'Notes', 'Address ID',
       'Primary Phone Number', 'Secondary Phone Number'],
      dtype='object')

Let's drop the "Notes" column for now and then fill them in later when creating reviews.

In [2148]:
local_resources_df.drop(columns=['Notes'], inplace=True)

In [2149]:
local_resources_df.rename(columns={'ID': 'id', 'Contact First Name': 'contact_first_name', 'Contact Last Name': 'contact_last_name', 'Company': 'company_name', \
                                   'Email': 'email_address', 'Preferred Contact Method': 'preferred_contact_method', \
                                   'Website': 'website', 'Address ID': 'address_id', 'Primary Phone Number': 'primary_phone_number',\
                                   'Secondary Phone Number': 'secondary_phone_number'}, inplace=True)

## Add availability IDs

In [2150]:
num_rows = local_resources_df['id'].count()

In [2151]:
local_resources_df.insert(10,"availability_id", range(len(volunteer_df) + 1, len(volunteer_df) + len(local_resources_df) + 1), True)

In [2152]:
local_resources_df.head()

,id,contact_first_name,contact_last_name,company_name,email_address,preferred_contact_method,website,address_id,primary_phone_number,secondary_phone_number,availability_id
0,0,NaN,NaN,Ace Security,NaN,Phone,NaN,197,6072738840,None,148
1,1,NaN,NaN,ACME Pest Control,NaN,Phone,NaN,198,607-844-8689,None,149
2,2,NaN,NaN,Bailey Construction,NaN,Phone,www.baileyconstruction.vpweb.com,199,607-209-4114,None,150
3,3,NaN,NaN,Bell's Auto Service,NaN,Phone,www.bellsauto.com,200,607-273-9325,None,151
4,4,NaN,NaN,Bumblebee Painters,info@bumblebeepainters.com,Email,www.bumblebeepainters.com,201,607-273-6521,None,152


In [2153]:
for row in local_resources_df.iterrows():
    print(dict(row[1]))
    break

{'id': 0, 'contact_first_name': nan, 'contact_last_name': nan, 'company_name': 'Ace Security', 'email_address': nan, 'preferred_contact_method': 'Phone', 'website': nan, 'address_id': 197, 'primary_phone_number': '6072738840', 'secondary_phone_number': None, 'availability_id': 148}


## Write the table out to a CSV file

In [2154]:
local_resources_df.to_csv('../out/local_resources.csv', index=False)